- To visualize the unit activations from CNN layers.

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load sample image set
sam_img = np.load('Val_data.npz')['val_X'] # Sample validation data as an exmaple.
sam_img_num = sam_img.shape[0]

In [ ]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((1,1,3))
def img_proc(sam_img):
    imgs_proc = np.zeros(sam_img.shape)
    for idx, img in enumerate(sam_img):
        img = np.asarray(img).astype('float32')
        img = img - vgg_mean
        imgs_proc[idx] = img[:,:,::-1]
    return imgs_proc

def get_layer_rep(li,imgs,method=None):
    n_imgs = imgs.shape[0]
    op = graph.get_tensor_by_name(relus[li]+':0')
    ft_sh = (n_imgs, op.shape[1], op.shape[2], op.shape[3])
    rep_ = np.zeros((ft_sh),dtype='float32')
    for i,img in enumerate(imgs):
        img = np.expand_dims(img,0)
        rp = sess.run(op, {x:img, is_training:False})
        rep_[i] = rp
    rep_ = rep_.reshape(n_imgs,-1)
    return rep_

def scale_01(x):
    rg = (np.max(x) - np.min(x))
    st = x - np.min(x)
    return st / rg

In [ ]:
# Load model
sess = tf.Session()
saver = tf.train.import_meta_graph('net-55.ckpt.meta')
saver.restore(sess, 'net-55.ckpt')
graph = tf.get_default_graph()
# Load placeholder
x = graph.get_tensor_by_name("Placeholder:0") 
is_training = graph.get_tensor_by_name("Placeholder_2:0")
# Get relu layer
relus = [op.name for op in graph.get_operations() if op.type=='Relu']
relus = relus[:14] + ['fc8/Conv2D']

In [ ]:
# Labels for t-SNE plot
labels = []
for ci in range(16):
    for si in range(10): # Number of sample
        labels.append(ci)

In [ ]:
# Plotting parameters
color_list = ['#E07F81','#AC84C8','#97C9EC','#7CBC75','#EDEC2E','#E1A822','#DA592C','#8A319B','#3D59B1','#6294D3','#82BEA6','#4C888E','#418335','#97C225','#9A9A33','#724821']
tlist = [] # Title list
for li in range(1,14):
    tlist.append('Conv '+str(li))
tlist += ['FC','Output']
label_list = ['Zero', 'One', 'Two', 'Three', 'Four', 'Five', 'Six',
              'Seven', 'Eight', 'Nine', 'Bed', 'Bird', 'Cat', 'Dog',
              'House', 'Tree']

In [ ]:
depth = 15 # CNN model depth
sam_img_proc = img_proc(sam_img) # Image preprocessing
for li in range(depth):
    features = get_layer_rep(li,sam_img_proc) 
    features = features.reshape(sam_img_num,-1)
    tsne = TSNE(n_components=2,perplexity=30).fit_transform(features) # You can tune perplexity.

    tx = tsne[:,0]
    ty = tsne[:,1]

    tx = scale_01(tx)
    ty = scale_01(ty)
    fig = plt.figure(figsize=(8,8))
    plt.title(tlist[li],fontsize=20)

    for ci in range(16):
        indices = [i for i, l in enumerate(labels) if l == ci]
        curr_tx = np.take(tx, indices)
        curr_ty = np.take(ty, indices)
        plt.scatter(curr_tx, curr_ty, c=color_list[ci], label=label_list[ci])
    plt.axis('off')
    plt.show()
    plt.close()

In [ ]:
# Raw input data features
imglen = 224*224*3
features = sam_img_proc.reshape(sam_img_num,imglen).copy()
tsne = TSNE(n_components=2,perplexity=30).fit_transform(features)

tx = tsne[:,0]
ty = tsne[:,1]

tx = scale_01(tx)
ty = scale_01(ty)
fig = plt.figure(figsize=(8,8))
plt.title('Input',fontsize=15)

for ci in range(16):
    indices = [i for i, l in enumerate(labels) if l == ci]
    curr_tx = np.take(tx, indices)
    curr_ty = np.take(ty, indices)
    plt.scatter(curr_tx, curr_ty, c=color_list[ci], label=label_list[ci])
plt.axis('off')
plt.show()
plt.close()